# 作業：使用 Flask 操作 Mongo

# [作業目標]
* 能夠使用 python 操作 mongodb 的增刪改查。
* API 設計上能符合 Restful 風格。

# [作業重點]
* 能使用正確的 Http Method 去接收 Request。
* 實作之前，Mongo Server 要記得開啟。
* 若 DB 和 Collection 原本不存在，請先用 Robo3T 手動建立。

# [本作業你可能需要使用的 packages]
* from pymongo import MongoClient
* from flask import Flask, request
* from bson.objectid import ObjectId

# 作業 1
* 使用 Post，傳入參數 name, age, email 參數，並把資料存入 Member Collection。

In [ ]:
from pymongo import MongoClient
from flask import Flask, request


# 連線 sever
client = MongoClient(host='127.0.0.1', port=27017)
# 連接資料庫
db = client['mydb']
# 指定集合
collection = db['member']

app = Flask(__name__)

@app.route("/member-management/members", methods=['POST'])
def add_member():
    name = request.form.get("name")
    age = request.form.get("age")
    email = request.form.get("email")
    
    result = db.member.insert_one({'name': name, 'age': age, 'email': email})
    return str(result.inserted_id)

if __name__ == "__main__":
    app.run()

# 作業 2
* 使用 Get，傳入參數 id 參數，並成功從 Mongo 中取得前面新增進去的資料。

In [ ]:
from pymongo import MongoClient
from flask import Flask, request, jsonify
from bson.objectid import ObjectId


# 連線 sever
client = MongoClient(host='127.0.0.1', port=27017)
# 連接資料庫
db = client['mydb']
# 指定集合
collection = db['member']

app = Flask(__name__)

@app.route("/member-management/members")
@app.route("/member-management/members/<id>", methods=['GET'])
def get_member(id=None):

    result = []
    
    if id is None: # 若沒有傳入特定 id，則回傳所有找到的資料
        members = db.member.find({})
        for member in members:
            member['_id'] = str(member['_id'])
            result.append(member)
    else:
        result = db.member.find_one({'_id': ObjectId(id)})
        if result is not None:
            result['_id'] = str(result['_id'])
            
    return jsonify(result)

if __name__ == "__main__":
    app.run()

# 作業 3
* 使用 PUT，傳入參數 id, name, age, email 參數，並修改 Mongo 中某一筆資料的值。

In [ ]:
from pymongo import MongoClient
from flask import Flask, request, jsonify
from bson.objectid import ObjectId


# 連線 sever
client = MongoClient(host='127.0.0.1', port=27017)
# 連接資料庫
db = client['mydb']
# 指定集合
collection = db['member']

app = Flask(__name__)

@app.route("/member-management/members/<id>", methods=['PUT'])
def update_member(id):
    
    result = 0
    
    name = request.form.get("name")
    age = request.form.get("age")
    email = request.form.get("email")
    
    new_member_value = { "$set": { "name": name, "age": age, "email":email } }
    
    upd_result = db.member.update_one({"_id": ObjectId(id)}, new_member_value)
    
    if upd_result is not None:
        result = upd_result.modified_count
    
    return "Update %s row." % str(result)

if __name__ == "__main__":
    app.run()

# 作業 4
* 使用 DELETE，傳入參數 id，並刪除 Mongo 中某一筆資料。

In [ ]:
from pymongo import MongoClient
from flask import Flask, request, jsonify
from bson.objectid import ObjectId


# 連線 sever
client = MongoClient(host='127.0.0.1', port=27017)
# 連接資料庫
db = client['mydb']
# 指定集合
collection = db['member']

app = Flask(__name__)

@app.route("/member-management/members/<id>", methods=['DELETE'])
def remove_member(id):
    
    result = 0
    
    del_result = db.member.delete_one({"_id": ObjectId(id)})
    
    if del_result is not None:
        result = del_result.deleted_count
    
    return "Delete %s row." % str(result)

if __name__ == "__main__":
    app.run()